In [1]:
from flask import Flask, request, jsonify
from datetime import datetime
import json
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from model import set_model, predict
from path import path

In [2]:
import json

In [3]:
def toJson(dir, dict):
    with open(dir, 'w', encoding='utf-8') as file:
        json.dump(dict, file, ensure_ascii=False, indent='\t')

def loadJson(dir):
    with open(dir, 'r', encoding='utf-8') as file:
        return json.load(file)

In [27]:
userId='musiclove529'
dir = path + 'data/' + userId
score = pd.read_csv(dir + '/output_sli_rec.txt',sep='\t',header=None,dtype=float,names=['score'])
test = pd.read_csv(dir + '/test', sep='\t',header=None,dtype=str,names=['label', 'uId', 'pId', 'cate', 'time', 'pHis', 'cHis', 'tHis'])
df = pd.concat([score, test], axis=1)
df = df.loc[df['score'].sort_values(ascending=False).index]
df = df.reset_index().drop(['index'], axis=1)
df = df[df['score']>0.5]
df = df[['score','pId','cate']]
df

,score,pId,cate
0,0.906994,6502,55B
1,0.905170,2755,41B
2,0.865847,1076,2B
3,0.841939,11024,4B
4,0.833174,10801,2B
5,0.826550,14723,55B
6,0.817342,5717,4B
7,0.816484,1977,55B
8,0.814642,2438,2B
9,0.814597,8393,55B


In [28]:
problemDF = pd.read_csv('problemInfo.csv', index_col=0, dtype=str)
problemDF = problemDF[['pId','successCnt']]
problemDF

,pId,successCnt
0,16156,69
1,16399,87
2,10729,18
3,21036,8
4,10335,44
...,...,...
9568,20352,392
9569,20215,391
9570,22015,387
9571,17863,958


In [29]:
df = pd.merge(df, problemDF, how='left', on='pId')
df

,score,pId,cate,successCnt
0,0.906994,6502,55B,1064
1,0.905170,2755,41B,1212
2,0.865847,1076,2B,7698
3,0.841939,11024,4B,1317
4,0.833174,10801,2B,3021
5,0.826550,14723,55B,579
6,0.817342,5717,4B,3717
7,0.816484,1977,55B,8953
8,0.814642,2438,2B,100099
9,0.814597,8393,55B,90553


In [30]:
scoreList = list(df['score'])
newScoreList = []
for s in scoreList:
    newScoreList.append(float("{:.1f}".format(s)))
newScoreList

[0.9,
 0.9,
 0.9,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.7,
 0.7,
 0.7,
 0.7,
 0.7,
 0.7,
 0.7,
 0.7,
 0.7,
 0.7,
 0.7,
 0.7,
 0.6,
 0.6,
 0.6,
 0.6,
 0.6,
 0.6,
 0.6,
 0.6,
 0.6,
 0.6,
 0.6,
 0.5,
 0.5,
 0.5,
 0.5]

In [31]:
df['score'] = newScoreList
df

,score,pId,cate,successCnt
0,0.9,6502,55B,1064
1,0.9,2755,41B,1212
2,0.9,1076,2B,7698
3,0.8,11024,4B,1317
4,0.8,10801,2B,3021
5,0.8,14723,55B,579
6,0.8,5717,4B,3717
7,0.8,1977,55B,8953
8,0.8,2438,2B,100099
9,0.8,8393,55B,90553


In [32]:
df = df.astype({'successCnt':'int'})
df.dtypes

score         float64
pId            object
cate           object
successCnt      int32
dtype: object

In [33]:
df = df.sort_values(['score','successCnt'], ascending=[False,False], ignore_index=True)
df

,score,pId,cate,successCnt
0,0.9,1076,2B,7698
1,0.9,2755,41B,1212
2,0.9,6502,55B,1064
3,0.8,2438,2B,100099
4,0.8,1330,41B,93491
5,0.8,8393,55B,90553
6,0.8,2443,2B,15307
7,0.8,1977,55B,8953
8,0.8,10886,41B,8446
9,0.8,5717,4B,3717


In [34]:
tagCnt = {}
problems = loadJson(path + 'problemData.json')
recDict = []
for i in range(len(df)):
    data = df.loc[i]
    id = data['pId']
    tag = data['cate']
    keys = tagCnt.keys()
    if tag in keys:
        tagCnt[tag] += 1
    else:
        tagCnt[tag] = 1
    if tagCnt[tag]<4:
        for p in problems:
            if int(id) == p['pId']:
                recDict.append(p)
    if len(recDict) == 20:
        break
recDict
    

        
    

[{'pId': 1076,
  'title': '저항',
  'successCnt': 7698,
  'tags': [{'tagName': '구현',
    'engName': 'Implementation',
    'url': 'https://www.acmicpc.net/problem/tag/102'}],
  'level': 'Bronze2',
  'url': 'https://www.acmicpc.net/problem/1076'},
 {'pId': 2755,
  'title': '이번학기 평점은 몇점?',
  'successCnt': 1212,
  'tags': [{'tagName': '수학',
    'engName': 'Mathematics',
    'url': 'https://www.acmicpc.net/problem/tag/124'},
   {'tagName': '구현',
    'engName': 'Implementation',
    'url': 'https://www.acmicpc.net/problem/tag/102'},
   {'tagName': '사칙연산',
    'engName': 'Arithmetic',
    'url': 'https://www.acmicpc.net/problem/tag/121'}],
  'level': 'Bronze1',
  'url': 'https://www.acmicpc.net/problem/2755'},
 {'pId': 6502,
  'title': '동혁 피자',
  'successCnt': 1064,
  'tags': [{'tagName': '수학',
    'engName': 'Mathematics',
    'url': 'https://www.acmicpc.net/problem/tag/124'},
   {'tagName': '구현',
    'engName': 'Implementation',
    'url': 'https://www.acmicpc.net/problem/tag/102'},
   {'tagN

In [ ]:
rec = list(df[df['score']>0.5]['pId'])
problems = loadJson(path + 'problemData.json')
recDict = []
for r in rec:
    for p in problems:
        if int(r) == p['pId']:
            recDict.append(p)
recDict